In [24]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [25]:
data_one_big_hole = pd.read_excel('data/uniaxial_one_big_hole.xlsx')
scaler = MinMaxScaler()
X = data_one_big_hole.drop(columns=['d(psi)/d(I1)', 'd(psi)/d(I2)'])
y = data_one_big_hole.drop(columns=['d(psi)/d(I1)', 'I1', 'I2'])
X_train, X_test, y_train, y_test = \
                train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [15]:
clr = MLPRegressor(activation='tanh', hidden_layer_sizes=(100,))
clr.fit(X_train, y_train)

MLPRegressor(activation='tanh')

In [18]:
from sklearn.metrics import confusion_matrix  # noqa: E402

pred = clr.predict(X_test)
# print(confusion_matrix(y_test, pred))

In [37]:
X.shape[1]

2

In [72]:
from skl2onnx import convert_sklearn  # noqa: E402
from skl2onnx.common.data_types import FloatTensorType  # noqa: E402

initial_type = [("float_input", FloatTensorType([2, None]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("binn.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [73]:
import onnxruntime as rt  # noqa: E402

sess = rt.InferenceSession("binn.onnx")

print(f"input name='{sess.get_inputs()[0].name}' and shape={sess.get_inputs()[0].shape}")
print(f"output name='{sess.get_outputs()[0].name}' and shape={sess.get_outputs()[0].shape}")

input name='float_input' and shape=[2, None]
output name='variable' and shape=[2, 1]


In [93]:
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

import numpy  # noqa: E402

dir(sess)

pred_onx = sess.run([label_name], X_test)

ValueError: Required inputs (['float_input']) are missing from input feed (['I1', 'I2']).

In [92]:
clr.predict(X_test)

/home/mmbmgp/.local/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


ValueError: X has 2 features, but MLPRegressor is expecting 4 features as input.